# Laboratorio 2

- Brando Reyes
- Juan Pablo Solis

## Task 1 - Preguntas Teoricas

1. ¿Por qué el modelo de Naive Bayes se le considera "naive"?
    - Este modelo se considera "Naive" debido a que asume que todas las características son independientes entre sí, lo cual rara vez es cierto en problemas del mundo real
2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo?
3. Investigue sobre Random Forest y responda:
    -  ¿Qué tipo de ensemble learning es este modelo?
        - Random Forest es un modelo de bagging el cual combina múltiples modelos que se entrenan de manera independiente utilizando diferentes subconjuntos de datos, y luego promedia o vota para obtener la predicción final.
    - ¿Cuál es la idea general detrás de Random Forest?
        - La idea general de Random Forest es combinar múltiples árboles de decisión para mejorar la precisión y reducir el riesgo de sobreajuste. 
    - ¿Por qué se busca baja correlación entre los árboles de Random Forest?
        - La baja correlación entre los árboles es clave para el éxito de Random Forest porque aumenta su capacidad de generalización. Si los árboles son altamente correlacionados, cometerán los mismos errores, lo que limitaría la mejora obtenida al combinarlos

# Task 2 Lectura y limpieza de archivo

In [26]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score

In [12]:
# Leer el archivo y cargarlo en un DataFrame
def cargar_datos(ruta_archivo):
    mensajes = []
    etiquetas = []
    
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        for linea in archivo:
            if "\t" in linea:
                etiqueta, mensaje = linea.strip().split("\t", 1)
                etiquetas.append(etiqueta.lower())
                mensajes.append(mensaje.lower())  # Convertir a minúsculas para uniformidad
    
    return pd.DataFrame({'etiqueta': etiquetas, 'mensaje': mensajes})


# Procesar los datos
ruta_archivo = 'entrenamiento.txt'  # Cambiar por la ruta correcta del archivo
df = cargar_datos(ruta_archivo)

print(df.head())

# Verificar la distribución de clases
print("\nDistribución de clases:")
print(df["etiqueta"].value_counts())


  etiqueta                                            mensaje
0      ham  go until jurong point, crazy.. available only ...
1      ham                      ok lar... joking wif u oni...
2     spam  free entry in 2 a wkly comp to win fa cup fina...
3      ham  u dun say so early hor... u c already then say...
4      ham  nah i don't think he goes to usf, he lives aro...

Distribución de clases:
etiqueta
ham     4818
spam     747
Name: count, dtype: int64


In [13]:
df.info

<bound method DataFrame.info of      etiqueta                                            mensaje
0         ham  go until jurong point, crazy.. available only ...
1         ham                      ok lar... joking wif u oni...
2        spam  free entry in 2 a wkly comp to win fa cup fina...
3         ham  u dun say so early hor... u c already then say...
4         ham  nah i don't think he goes to usf, he lives aro...
...       ...                                                ...
5560     spam  this is the 2nd time we have tried 2 contact u...
5561      ham               will ü b going to esplanade fr home?
5562      ham  pity, * was in mood for that. so...any other s...
5563      ham  the guy did some bitching but i acted like i'd...
5564      ham                         rofl. its true to its name

[5565 rows x 2 columns]>

### Limpiar el dataset

In [14]:

# Función para limpiar el texto
def limpiar_mensaje(mensaje):

    # Eliminar caracteres especiales excepto letras, números y espacios
    mensaje = re.sub(r"[^a-zA-Z0-9\s]", '', mensaje)
    # Quitar espacios adicionales
    mensaje = re.sub(r'\s+', ' ', mensaje).strip()
    # Convertir a minúsculas
    return mensaje.lower()

#Aplicar la limpieza al DataFrame
df['mensaje'] = df['mensaje'].apply(limpiar_mensaje)

# Verificar los resultados
print(df.head())


  etiqueta                                            mensaje
0      ham  go until jurong point crazy available only in ...
1      ham                            ok lar joking wif u oni
2     spam  free entry in 2 a wkly comp to win fa cup fina...
3      ham        u dun say so early hor u c already then say
4      ham  nah i dont think he goes to usf he lives aroun...


## Dividir el dataset

In [15]:

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['etiqueta'])

# Verificar las formas de los conjuntos
print("Tamaño del conjunto de entrenamiento:", train_df.shape)
print("Tamaño del conjunto de prueba:", test_df.shape)

# Opcional: Verificar la distribución de clases en ambos conjuntos
print("\nDistribución en el conjunto de entrenamiento:")
print(train_df['etiqueta'].value_counts(normalize=True))

print("\nDistribución en el conjunto de prueba:")
print(test_df['etiqueta'].value_counts(normalize=True))


Tamaño del conjunto de entrenamiento: (4452, 2)
Tamaño del conjunto de prueba: (1113, 2)

Distribución en el conjunto de entrenamiento:
etiqueta
ham     0.865678
spam    0.134322
Name: proportion, dtype: float64

Distribución en el conjunto de prueba:
etiqueta
ham     0.866128
spam    0.133872
Name: proportion, dtype: float64


## Construccion del modelo

In [23]:
# Función para entrenar el modelo basado en Bayes con Laplace Smoothing 
def entrenar_bayes(mensajes, etiquetas, alpha=1.0):
    # Inicializar contadores
    total_mensajes = len(mensajes)
    total_por_categoria = {}
    palabras_por_categoria = {}
    total_palabras_por_categoria = {}
    vocabulario = set()
    
    # Procesar cada mensaje
    for i in range(len(mensajes)):
        etiqueta = etiquetas[i]
        mensaje = limpiar_mensaje(mensajes[i])  # Llamar a la función de limpieza
        palabras = mensaje.split()
        
        # Inicializar estructuras para la categoría si no existen
        if etiqueta not in total_por_categoria:
            total_por_categoria[etiqueta] = 0
            palabras_por_categoria[etiqueta] = {}
            total_palabras_por_categoria[etiqueta] = 0
        
        # Actualizar contadores
        total_por_categoria[etiqueta] += 1
        total_palabras_por_categoria[etiqueta] += len(palabras)
        
        for palabra in palabras:
            vocabulario.add(palabra)
            if palabra not in palabras_por_categoria[etiqueta]:
                palabras_por_categoria[etiqueta][palabra] = 0
            palabras_por_categoria[etiqueta][palabra] += 1
    
    # Calcular probabilidades con Laplace Smoothing
    tamaño_vocabulario = len(vocabulario)
    probabilidad_categoria = {cat: total_por_categoria[cat] / total_mensajes for cat in total_por_categoria}
    probabilidad_palabra_dado_categoria = {}
    
    for categoria in palabras_por_categoria:
        probabilidad_palabra_dado_categoria[categoria] = {}
        for palabra in vocabulario:
            conteo_palabra = palabras_por_categoria[categoria].get(palabra, 0)
            probabilidad_palabra_dado_categoria[categoria][palabra] = (conteo_palabra + alpha) / (total_palabras_por_categoria[categoria] + alpha * tamaño_vocabulario)
    
    return probabilidad_categoria, probabilidad_palabra_dado_categoria, tamaño_vocabulario, vocabulario

# Función para predecir la categoría de un mensaje
def predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=1.0):
    mensaje = limpiar_mensaje(mensaje)  # Llamar a la función de limpieza antes de predecir
    palabras = mensaje.split()
    probabilidades = {}
    
    for categoria in prob_categoria:
        # Iniciar con la probabilidad de la categoría
        probabilidad = prob_categoria[categoria]
        
        # Multiplicar las probabilidades de las palabras
        for palabra in palabras:
            if palabra in prob_palabra_categoria[categoria]:
                probabilidad *= prob_palabra_categoria[categoria][palabra]
            else:
                # Manejar palabras desconocidas con Laplace Smoothing
                probabilidad *= alpha / (tamaño_vocabulario + alpha)
        
        probabilidades[categoria] = probabilidad
    
    # Devolver la categoría con mayor probabilidad
    return max(probabilidades, key=probabilidades.get)

# Entrenar el modelo usando el conjunto de entrenamiento
X_train = train_df['mensaje'].tolist()  # Convertir a lista si es necesario
y_train = train_df['etiqueta'].tolist()
prob_categoria, prob_palabra_categoria, tamaño_vocabulario, vocabulario = entrenar_bayes(X_train, y_train, alpha=0.5)

# Imprimir estadísticas del modelo entrenado
print("Tamaño del vocabulario:", tamaño_vocabulario)

# Probar el modelo con el conjunto de prueba
X_test = test_df['mensaje'].tolist()
y_test = test_df['etiqueta'].tolist()

correctos = 0
for i in range(len(X_test)):
    mensaje = X_test[i]
    etiqueta_real = y_test[i]
    etiqueta_predicha = predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=0.5)
    
    if etiqueta_predicha == etiqueta_real:
        correctos += 1

# Calcular la precisión
precision = correctos / len(X_test)
print(f"Precisión del modelo: {precision:.2f}")


Tamaño del vocabulario: 8353
Precisión del modelo: 0.98


### Prueba Con metrica Recall

- Creemos que recall es una metrica adecuada debido a que permite minimizar los falsos negativos. En este caso al tener spam como una clase minoritaria  queremos asegurarnos de que el modelo pueda identificar correctamente la mayor cantidad de mensajes spam

In [24]:
# Función para calcular el Recall por clase
def calcular_recall(y_real, y_pred, clase_objetivo):
    verdaderos_positivos = 0
    falsos_negativos = 0

    for i in range(len(y_real)):
        if y_real[i] == clase_objetivo:
            if y_pred[i] == clase_objetivo:
                verdaderos_positivos += 1
            else:
                falsos_negativos += 1
    
    # Recall = Verdaderos Positivos / (Verdaderos Positivos + Falsos Negativos)
    if (verdaderos_positivos + falsos_negativos) == 0:
        return 0  # Evitar división por cero
    return verdaderos_positivos / (verdaderos_positivos + falsos_negativos)

# Evaluar el modelo en el conjunto de entrenamiento
y_train_pred = [predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=0.5) for mensaje in X_train]
recall_train_spam = calcular_recall(y_train, y_train_pred, clase_objetivo="spam")
recall_train_ham = calcular_recall(y_train, y_train_pred, clase_objetivo="ham")
recall_train_promedio = (recall_train_spam + recall_train_ham) / 2

# Evaluar el modelo en el conjunto de prueba
y_test_pred = [predecir_bayes_flexible(mensaje, prob_categoria, prob_palabra_categoria, tamaño_vocabulario, alpha=0.5) for mensaje in X_test]
recall_test_spam = calcular_recall(y_test, y_test_pred, clase_objetivo="spam")
recall_test_ham = calcular_recall(y_test, y_test_pred, clase_objetivo="ham")
recall_test_promedio = (recall_test_spam + recall_test_ham) / 2

# Presentar métricas de desempeño con el formato requerido
print("=== Entrenamiento manual  ===\n")

print("=== Evaluación en conjunto de entrenamiento ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_train_spam:.3f}")
print(f"- Recall para ham: {recall_train_ham:.3f}")
print(f"- Recall promedio: {recall_train_promedio:.3f}\n")

print("=== Evaluación en conjunto de prueba ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_test_spam:.3f}")
print(f"- Recall para ham: {recall_test_ham:.3f}")
print(f"- Recall promedio: {recall_test_promedio:.3f}")


=== Entrenamiento manual  ===

=== Evaluación en conjunto de entrenamiento ===

Métricas de evaluación:
- Recall para spam: 0.975
- Recall para ham: 0.997
- Recall promedio: 0.986

=== Evaluación en conjunto de prueba ===

Métricas de evaluación:
- Recall para spam: 0.886
- Recall para ham: 0.995
- Recall promedio: 0.940


##  2.3 Mensajes futuros

In [25]:
# Bucle para entrada de mensajes en consola hasta que el usuario escriba "z"
print("\n=== Clasificación interactiva de mensajes ===")
print("Ingrese un mensaje para clasificarlo. Escriba 'z' para salir.\n")

while True:
    mensaje_usuario = input("Mensaje: ").strip()
    
    if mensaje_usuario.lower() == "z":
        print("Saliendo del clasificador...")
        break  # Detener la ejecución cuando el usuario escriba 'z'
    
    palabras = mensaje_usuario.split()
    probabilidades = {}

    # Calcular las probabilidades de spam y ham
    for categoria in prob_categoria:
        probabilidad = prob_categoria[categoria]  # P(Categoria)
        
        for palabra in palabras:
            if palabra in prob_palabra_categoria[categoria]:
                probabilidad *= prob_palabra_categoria[categoria][palabra]
            else:
                # Manejar palabras desconocidas con Laplace Smoothing
                probabilidad *= 0.5 / (tamaño_vocabulario + 0.5)
        
        probabilidades[categoria] = probabilidad

    # Normalizar las probabilidades para obtener una distribución entre 0 y 1
    suma_probabilidades = sum(probabilidades.values())
    if suma_probabilidades > 0:
        for categoria in probabilidades:
            probabilidades[categoria] /= suma_probabilidades

    # Determinar la categoría con mayor probabilidad
    categoria_predicha = max(probabilidades, key=probabilidades.get)

    # Mostrar resultados en consola
    print("\n--- Resultados de clasificación ---")
    print(f"Mensaje ingresado: \"{mensaje_usuario}\"")
    print(f"Probabilidad de SPAM: {probabilidades['spam']:.3f}")
    print(f"Probabilidad de HAM: {probabilidades['ham']:.3f}")
    print(f"Clasificación: {categoria_predicha.upper()}\n")



=== Clasificación interactiva de mensajes ===
Ingrese un mensaje para clasificarlo. Escriba 'z' para salir.


--- Resultados de clasificación ---
Mensaje ingresado: "Hello, you have won a brand new car! Just click the following link"
Probabilidad de SPAM: 1.000
Probabilidad de HAM: 0.000
Clasificación: SPAM


--- Resultados de clasificación ---
Mensaje ingresado: "Dave did you finish your homework?"
Probabilidad de SPAM: 0.001
Probabilidad de HAM: 0.999
Clasificación: HAM


--- Resultados de clasificación ---
Mensaje ingresado: "What time is it?"
Probabilidad de SPAM: 0.040
Probabilidad de HAM: 0.960
Clasificación: HAM

Saliendo del clasificador...


## 2.4 Comparacion de librerias

In [27]:


# Convertir los mensajes en formato numérico usando Bag of Words
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)  # Ajustar y transformar training
X_test_vectorized = vectorizer.transform(X_test)  # Solo transformar testing

# Entrenar el modelo Naïve Bayes con sklearn
modelo_sklearn = MultinomialNB(alpha=0.5)  # Alpha para Laplace Smoothing
modelo_sklearn.fit(X_train_vectorized, y_train)

# Predicciones en training y testing
y_train_pred_sklearn = modelo_sklearn.predict(X_train_vectorized)
y_test_pred_sklearn = modelo_sklearn.predict(X_test_vectorized)

# Calcular Recall en training y testing
recall_train_spam_sklearn = recall_score(y_train, y_train_pred_sklearn, pos_label="spam")
recall_train_ham_sklearn = recall_score(y_train, y_train_pred_sklearn, pos_label="ham")
recall_train_promedio_sklearn = (recall_train_spam_sklearn + recall_train_ham_sklearn) / 2

recall_test_spam_sklearn = recall_score(y_test, y_test_pred_sklearn, pos_label="spam")
recall_test_ham_sklearn = recall_score(y_test, y_test_pred_sklearn, pos_label="ham")
recall_test_promedio_sklearn = (recall_test_spam_sklearn + recall_test_ham_sklearn) / 2

# Presentar métricas de desempeño con el mismo formato que en la implementación manual
print("=== Evaluación con Naïve Bayes de sklearn ===\n")

print("=== Evaluación en conjunto de entrenamiento ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_train_spam_sklearn:.3f}")
print(f"- Recall para ham: {recall_train_ham_sklearn:.3f}")
print(f"- Recall promedio: {recall_train_promedio_sklearn:.3f}\n")

print("=== Evaluación en conjunto de prueba ===\n")
print("Métricas de evaluación:")
print(f"- Recall para spam: {recall_test_spam_sklearn:.3f}")
print(f"- Recall para ham: {recall_test_ham_sklearn:.3f}")
print(f"- Recall promedio: {recall_test_promedio_sklearn:.3f}")


=== Evaluación con Naïve Bayes de sklearn ===

=== Evaluación en conjunto de entrenamiento ===

Métricas de evaluación:
- Recall para spam: 0.977
- Recall para ham: 0.997
- Recall promedio: 0.987

=== Evaluación en conjunto de prueba ===

Métricas de evaluación:
- Recall para spam: 0.893
- Recall para ham: 0.994
- Recall promedio: 0.943


### ¿Cuál implementación lo hizo mejor? ¿Su implementación o la de la librería?
    - Ambas pruebas de recall obtuvieron valores similares
        - La implementacion con Sklearn tiene mejor Recall para Spam (+0.007), lo que significa que detecta más mensajes spam correctamente.
        - Manual tiene mejor Recall para Ham (+0.001), aunque la diferencia es mínima.
        - Sklearn tiene mejor Recall Promedio (+0.003), lo que indica un mejor balance en ambas clases.
### ¿Por qué cree que se debe esta diferencia?
    - La principal diferencia es que sklearn maneja mejor los cálculos numéricos usando logaritmos en lugar de multiplicaciones, evitando errores de precisión y subdesbordamiento. Además, CountVectorizer preprocesa mejor el texto, eliminando ruido y mejorando la representación del mensaje